In [3]:
import os
import exploface
import explibrosa
import emrecdem as erd

In [4]:
os.getcwd()

'/home/vincent/emotion/emrecdem-framework'

In [5]:
# change dir to root
os.chdir('../.')
os.getcwd()

'/home/vincent/emotion'

### Specify data location

In [6]:
emrecdemStudyDataFolder = "/media/sf_sharedfolder/Emotion/emrecdemstudydata"
openface_outputfolder = emrecdemStudyDataFolder + "/OpenFaceOutput"

### Identify csv files

In [7]:
# Search for files with csv extension, because we are only interested in those
# This assumes that there are no other csv files in folder other than the ones produced by OpenFace.
from os import listdir

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

filenames = find_csv_filenames(openface_outputfolder) # filenames will be a list

In [8]:
filenames

['P18_S2_IAPS_HAPPY_Cfront.csv',
 'P18_S2_IAPS_SAD_Cfront.csv',
 'P21_S2_IAPS_HAPPY_C1.csv',
 'P21_S2_IAPS_SAD_C1.csv']

In [9]:
#https://www.dataquest.io/blog/python-pandas-databases/
databasefile = emrecdemStudyDataFolder + "/emrecdem.db"

### Create database

In [10]:
erd.features2sqlitedb.createdatabase(databasefile = databasefile)
# Now on command line: sqlite3 pathtomydatabase.db
# should give access to database
# and .tables should show overview of the tables as created

In [11]:
deleteDataBaseEntriesOnFirstRun = True

### Loop over files, extract features, store features in database

In [13]:
for i in range(len(filenames)):
    # select one file from list of filenames
    openface_file = openface_outputfolder + '/' + filenames[i]
    # Find wav file that matches the csv file produced by OpenFace
    # based on the assumption that filenames are identical except from file extension
    audio_file = erd.findwav(filenames[i], emrecdemStudyDataFolder)
    print("generate video features")
    # process with exploface
    openface_features = exploface.get_feature_time_series(openface_file)
    print("generate audio features")
    # process with explibrosa
    # explibrosa.get_info(audio_file)
    audio_features = explibrosa.get_feature_time_series(audio_file)
    
    # extract participant id (PID) and experiment id (EXP) from filename:
    PID = filenames[i].split("_")[0].split("P")[1]
    EXP = ''.join(filenames[i].split("_")[1:4]) # extract integers from filename
    EXP = EXP.split(".csv")[0] # remove .csv at the end
    openface_features = erd.downsample(openface_features,res = '0.2S')
    openface_features = erd.tidyup_video_features(openface_features, PID, EXP)
    
    audio_features = erd.downsample(audio_features,res = '0.2S')
    audio_features = erd.tidyup_audio_features(audio_features, PID, EXP)
    
    #http://www.sqlitetutorial.net/sqlite-python/create-tables/
    #erd.features2sqlitedb.checkdbcontent(databasefile)
    # features to database
    if (deleteDataBaseEntriesOnFirstRun == 0):
        deleteDataBaseEntries = True
    else:
        deleteDataBaseEntries = False
    
    erd.features2sqlitedb.features2db(audio_features, openface_features,
            databasefile, deleteDataBaseEntries)
    # check database content
    erd.checkdbcontent(databasefile)
    
    print(openface_features.head())
    print(audio_features.head())
 

P18_S2_IAPS_HAPPY.wav
find /media/sf_sharedfolder/Emotion/emrecdemstudydata -name P18_S2_IAPS_HAPPY.wav
['/media/sf_sharedfolder/Emotion/emrecdemstudydata/P18/S2/Audio/P18_S2_IAPS_HAPPY.wav']


NameError: name 'os' is not defined